# Working Doc, not Report Doc

## Commenced 20 Aug 2018

## Steps to do

> 1. Import the two data files (predictors and targets) and merge, with pandas.
> 2. Cut down to the predictors to be used. 
> 3. Create composite variables.
> 4. Preliminary data exploration:
>> Categorical variables: check number of categories including blanks; create filled bar charts.<br>
>> Quantitative variables: get statistics such as mean, median, sd, range, # blanks; create violin or box plots.
> 5. Decide on further data mgt., such as: "rare" categories, binned categories, delete all rows with blanks/if too many blanks.
> 6. Train/test split.
> 7. Create decision tree as a baseline reference for random forest.
> 8. Create random forest. Tune parameters, consider boosting/bagging, confusion matrix, what is performance?
> 9. Predict on test set.
> 10. What do I now know? What does everything mean?

In [3]:
import pandas as pd
import numpy as np
import time
import matplotlib as plt
import seaborn as sns
import random

## Check uniqueness, no duplicate ids

In [4]:
# check select_data
select_df = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/select_data.csv')
select_df.shape

(59400, 23)

In [5]:
# check all are unique
select_df['id'].nunique()

59400

In [6]:
select_df.head()

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,69572,14/03/11,long_rainy,1390,Roman,34.938093,-9.856322,Lake Nyasa,11,109,...,1999,gravity,user-group,annually,soft,enough,spring,groundwater,communal standpipe,functional
1,8776,06/03/13,long_rainy,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,20,280,...,2010,gravity,user-group,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,34310,25/02/13,short_dry,686,World vision,37.460664,-3.821329,Pangani,21,250,...,2009,gravity,user-group,per bucket,soft,enough,dam,surface,communal standpipe,functional
3,67743,28/01/13,short_dry,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,90,58,...,1986,submersible,user-group,never pay,soft,dry,borehole,groundwater,communal standpipe,non functional
4,19728,13/07/11,long_dry,0,Artisan,31.130847,-1.825359,Lake Victoria,18,0,...,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,functional


select_data is ok. There is a problem with further data as shown here, Should also be 59400 unique id

In [7]:
df_check = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data03.csv')
# reindex for good luck
df_check.reset_index(drop=True, inplace=True)
df_check.shape

(59400, 24)

In [8]:
df_check['id'].value_counts()

73516    11
57806    11
72113    11
68019    11
46000    11
33218    11
35267    11
17326    11
61900    11
25514    11
20952    11
60347    11
18907    11
68487    11
66438    11
72581    11
29212    11
33314    11
37716    11
41510    11
35767    11
16794    11
8790     11
33058    11
11267    11
51467    11
6417     11
39925    11
41970    11
74006    11
         ..
48198     1
19544     1
21595     1
52328     1
19576     1
27772     1
25501     1
27548     1
19352     1
39810     1
39522     1
45671     1
60044     1
60076     1
68272     1
13015     1
49897     1
60140     1
58093     1
23290     1
31486     1
51976     1
56074     1
66321     1
70419     1
15126     1
21275     1
29503     1
25469     1
71683     1
Name: id, Length: 9400, dtype: int64

In [9]:
df_check['id'].nunique()

9400

In [10]:
counts = select_df['gps_height'].value_counts()
print("Zeros", counts[0], '\n')

Zeros 20438 



## Work out the id issue! Why are there 9,400 unique ids?

In [11]:
for col in select_df.columns: print(col)

id
date_recorded
season_recorded
gps_height
installer
longitude
latitude
basin
region_code
population
public_meeting
scheme_management
permit
construction_year
extraction_type_group
management_group
payment_type
water_quality
quantity_group
source_type
source_class
waterpoint_type_group
status_group


## Begin again


In [12]:
# need helper in dict to find key of maximum value, used in gps_helper
def find_dict_max(dict_):
    for keys, values in dict_.items():
        if values == max(dict_.values()):
            return keys

In [35]:
# N.B. .at, .iat can only take a single value
def gps_helper(df):
    """ 
    takes average height of Euclidian 3 nearest neighbours by distance (lat/long)
    trying every short cut to optimise
    Not even finding square root of error terms to save a calculation
    Notice that you still need to return a df_gps_list for later incorporation
    (Setting just not working by any means)
    
    This version works with putting new value directly into base case df
    
    """
    for i in df.index:
        # v just progess checking during running of function
        if i % 100 == 0:
#             print(i)
            pass
        
        if df.iat[i, 3] != 0: 
           pass
            
        else: 
            # neighbours = {index: distance, ...} - are dummy keys / values that get replaced
            neighbours = {999999: 9999, 999998: 9998, 999997: 9997}
            for j in df.index: 
                if j != i: 
                    euclidian_dist = ((df.iat[i, 6] - df.iat[j, 6])**2 + (df.iat[i, 5] - df.iat[j, 5])**2)
    
                    if euclidian_dist < max(neighbours.values()):
                        # if yes, need to remove max distance so far and update  with smaller distance
                        neighbours.pop(find_dict_max(neighbours))
                        neighbours.update({j: euclidian_dist})
 
            heights = []
            for k in neighbours.keys():
                heights.append(df['gps_height'][k])
            if len(heights) != 0:
                df.iat[i, 3] = sum(heights) / len(heights)
            else: 
                df.iat[i, 3] = 0

    zero_count = 0
    for m in range(df.shape[0]):
        if df.iat[m, 3] == 0:
            zero_count += 1

    print("number of zero heights is: {}".format(zero_count))
    return df


In [36]:
df = select_df

In [15]:
# template
start = time.time()
print("initial zeros", df['gps_height'].value_counts()[0])
df[:2000] = gps_helper(df[:2000])
counts = df['gps_height'].value_counts()
stop = time.time()
print("remaining zeros", df['gps_height'].value_counts()[0])
print(stop - start, "s")

initial zeros 20438
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
number of zero heights is: 640
remaining zeros 20399
33.747048139572144 s


In [37]:
# keep shuffling the df and redo function for for 1000 entries

In [107]:
print("initial zeros", df['gps_height'].value_counts()[0])
iters = []
start = time.time()
for i in range(60):
    df = df.reindex(np.random.permutation(df.index))
    df = df.reset_index(drop=True)
    df[:1000] = gps_helper(df[:1000])
    iters.append(i)
    print(i)

stop = time.time()
print("iterations", max(iters))
print(stop - start, "s")
print("remaining zeros", df['gps_height'].value_counts()[0])

initial zeros 4414
number of zero heights is: 55
0
number of zero heights is: 64
1
number of zero heights is: 81
2
number of zero heights is: 88
3
number of zero heights is: 73
4
number of zero heights is: 78
5
number of zero heights is: 70
6
number of zero heights is: 59
7
number of zero heights is: 67
8
number of zero heights is: 62
9
number of zero heights is: 67
10
number of zero heights is: 59
11
number of zero heights is: 69
12
number of zero heights is: 62
13
number of zero heights is: 73
14
number of zero heights is: 69
15
number of zero heights is: 73
16
number of zero heights is: 74
17
number of zero heights is: 74
18
number of zero heights is: 66
19
number of zero heights is: 66
20
number of zero heights is: 67
21
number of zero heights is: 62
22
number of zero heights is: 63
23
number of zero heights is: 59
24
number of zero heights is: 68
25
number of zero heights is: 65
26
number of zero heights is: 74
27
number of zero heights is: 71
28
number of zero heights is: 59
29
n

In [41]:
# check no corruption of ids (multiple same ids)

In [224]:
df['id'].nunique()

59400

In [59]:
# save to a file
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/gps_adjusted.csv', index=False)

In [51]:
# Redo for construction_year

In [52]:
def construction_helper(df):
    """ 
    BASED ON gps_helper function
    INPUT is the whole dataframe, so that you can easily traverse rows and columns
    OUTPUT is whole df that has the new construction_year
    IDEA: if construction_year is zero, iterate df to find matches for all chosen comparison variables.
    If all match, add the iterative row's year to a list and take average of list at end. Put that into 
    output df's entry.
   
    """
    for i in df.index:
        # v just progess checking during running of function
        if i % 1000 == 0:
#             print(i)
            pass
        
        if df.iat[i, 13] != 0: 
           pass
            
        else: 
            # initialise list that you will put matching years into
            matching_years = []
            for j in df.index: 
                # Many construction_year = 0, so not using these
                if (j != i) and (df.iat[j, 13] != 0):
                    # using a counter for readability
                    counter = 0
                    if df.iat[i, 4] == df.iat[j, 4]:
                        counter += 1
                    if df.iat[i, 11] == df.iat[j, 11]:
                        counter += 1 
                    if df.iat[i, 14] == df.iat[j, 14]:
                        counter += 1 
                    if df.iat[i, -1] == df.iat[j, -1]:
                        counter += 1    
                    # check if all items match
                    if counter == 4:
                        # if yes, add to matching_years
                        matching_years.append(df.iat[j, 13])
                    else:
                        pass
                    
            # now need to get average year of the matches   
            if len(matching_years) != 0:
                df.iat[i, 13] = round(sum(matching_years)/len(matching_years))
            else: 
                df.iat[i, 13] = 0

    zero_count = 0
    for m in df.index:
        if df.iat[m, 13] == 0:
            zero_count += 1
    
    print("final number of zero years is: {}".format(zero_count))
    return df

In [57]:
df['construction_year'].value_counts()[0]

20381

In [106]:
print("initial zeros", df['construction_year'].value_counts()[0])
iters = []
start = time.time()
for i in range(80):
    df = df.reindex(np.random.permutation(df.index))
    df = df.reset_index(drop=True)
    df[:1000] = construction_helper(df[:1000])
    iters.append(i)
    print(i)

stop = time.time()
print("iterations", max(iters))
print(stop - start, "s")
print("remaining zeros", df['construction_year'].value_counts()[0])

initial zeros 12941
0
final number of zero years is: 211
0
0
final number of zero years is: 205
1
0
final number of zero years is: 224
2
0
final number of zero years is: 234
3
0
final number of zero years is: 214
4
0
final number of zero years is: 215
5
0
final number of zero years is: 191
6
0
final number of zero years is: 231
7
0
final number of zero years is: 206
8
0
final number of zero years is: 220
9
0
final number of zero years is: 227
10
0
final number of zero years is: 206
11
0
final number of zero years is: 228
12
0
final number of zero years is: 196
13
0
final number of zero years is: 206
14
0
final number of zero years is: 209
15
0
final number of zero years is: 213
16
0
final number of zero years is: 210
17
0
final number of zero years is: 217
18
0
final number of zero years is: 235
19
0
final number of zero years is: 188
20
0
final number of zero years is: 211
21
0
final number of zero years is: 211
22
0
final number of zero years is: 216
23
0
final number of zero years i

In [109]:
# save to a file
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/construction_adjusted.csv', index=False)

### 4. Preliminary data exploration

Use Roland Jeannier's code.
https://medium.com/@rtjeannier/pandas-101-fbb5bf86a9bc

In [ ]:
def eda_helper(df):
   dict_list = []
   for col in df.columns:
       data = df[col]
       dict_ = {}
       # The null count for a column. 
       dict_.update({"null_count" : data.isnull().sum()})
       # Counting the unique values in a column
       dict_.update({"unique_count" : len(data.unique())})
       # Finding the types of data in the column
       # This is useful for finding out potential problems with type mismatches
       dict_.update({"data_type" : set([type(d).__name__ for d in data])})
       #dict_.update({"score" : match[1]})
       dict_list.append(dict_)
   eda_df = pd.DataFrame(dict_list)
   eda_df.index = df.columns
   eda_df.sort_values(by=['null_count', 'unique_count'], ascending=[True, False], inplace=True)
       
   return eda_df

In [ ]:
df = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data03.csv')
# reindex for good luck
df.reset_index(drop=True, inplace=True)
# run eda_helper
print(eda_helper(df))    

### 2B. Need to make most predictors into a categorical variables

In [ ]:
categories = {'installer', 'basin', 'region_code', 
             'population', 'public_meeting', 'scheme_management', 'permit',
             'extraction_type_group', 'management_group', 'payment_type', 
              'water_quality', 'quantity_group', 'source_type', 
             'source_class', 'waterpoint_type_group', 'status_group'}

for item in categories:
    select_data[item] = select_data[item].astype('category')
# check types
select_data.dtypes

**To Do:** Run construction_year_helper on the complete data set that you want to use. No need to run twice as seems to have no zero entries. (Shouldn't do by its construction.) 

### What can you see globally?

abc

In [ ]:
# Lost categorisation - need to redo.

In [ ]:
# Null counts only in some places.

In [ ]:
df.head()

In [60]:
df['id'].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
64138    1
57993    1
60040    1
33413    1
35460    1
45699    1
41601    1
43648    1
70263    1
68212    1
20442    1
23134    1
19036    1
29275    1
25177    1
27224    1
4695     1
6742     1
597      1
2644     1
10832    1
53839    1
55886    1
55950    1
53903    1
        ..
52531    1
64821    1
62836    1
58678    1
52595    1
50546    1
56689    1
54640    1
9582     1
15725    1
13676    1
3435     1
28007    1
25958    1
32101    1
30052    1
19811    1
23905    1
21856    1
68939    1
66890    1
73033    1
70984    1
42302    1
48445    1
46396    1
36155    1
34106    1
38200    1
0        1
Name: id, Length: 59400, dtype: int64

In [61]:
df['longitude'].value_counts()[0]

1812

In [62]:
df['latitude'].value_counts()

-2.000000e-08    1812
-6.975052e+00       2
-6.983115e+00       2
-6.989456e+00       2
-6.990731e+00       2
-2.516619e+00       2
-2.489378e+00       2
-3.342432e+00       2
-6.975965e+00       2
-1.793342e+00       2
-7.056923e+00       2
-6.978322e+00       2
-7.104625e+00       2
-7.056372e+00       2
-2.496459e+00       2
-2.501627e+00       2
-2.485227e+00       2
-2.494353e+00       2
-7.177203e+00       2
-6.981326e+00       2
-2.480043e+00       2
-6.978755e+00       2
-6.990549e+00       2
-6.990147e+00       2
-2.465246e+00       2
-6.981884e+00       2
-6.962475e+00       2
-6.959749e+00       2
-6.980122e+00       2
-7.095131e+00       2
                 ... 
-3.487229e+00       1
-3.305834e+00       1
-1.086099e+01       1
-4.265410e+00       1
-4.828176e+00       1
-4.750163e+00       1
-3.354840e+00       1
-3.411358e+00       1
-8.926957e+00       1
-5.435762e+00       1
-4.476671e+00       1
-5.726001e+00       1
-2.743032e+00       1
-9.646831e+00       1
-8.124530e

In [63]:
df['gps_height'].value_counts()[0]

4823

In [64]:
df['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
300      1476
100      1146
50       1139
500      1009
350       986
120       916
400       775
60        706
30        626
40        552
80        533
450       499
20        462
600       438
230       388
75        289
1000      278
800       269
90        265
130       264
25        255
320       249
35        245
360       222
140       215
        ...  
8848        1
628         1
4520        1
468         1
693         1
725         1
789         1
821         1
5300        1
3127        1
2345        1
3031        1
886         1
392         1
424         1
2807        1
726         1
694         1
2569        1
4788        1
662         1
4660        1
406         1
1032        1
1160        1
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [65]:
df['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [66]:
df['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [67]:
df['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

In [68]:
df['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [69]:
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [70]:
df['basin'].value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [71]:
df['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [72]:
df['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [73]:
df['season_recorded'].value_counts()

long_rainy     22242
short_dry      18756
long_dry       16432
short_rainy     1970
Name: season_recorded, dtype: int64

In [74]:
df['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

In [75]:
df['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [76]:
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [77]:
df['installer'].value_counts()

DWE                         17402
Government                   1825
RWE                          1206
Commu                        1060
DANIDA                       1050
KKKT                          898
Hesawa                        840
0                             777
TCRS                          707
Central government            622
CES                           610
Community                     553
DANID                         552
District Council              551
HESAWA                        539
World vision                  408
LGA                           408
WEDECO                        397
TASAF                         396
District council              392
Gover                         383
AMREF                         329
TWESA                         316
WU                            301
Dmdd                          287
ACRA                          278
World Vision                  270
SEMA                          249
DW                            246
OXFAM         

In [78]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [79]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [80]:
df['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [212]:
df['population'].value_counts()

0        21381
100      16391
1         7025
500       5741
1000      3947
50        1829
1000+     1690
30         804
20         592
Name: population, dtype: int64

In retrospect, below is too involved, can be simpler.

In [213]:
# population_needs bins
# look at sorting first to visually inspect potential bins
popn_df = df['population'].value_counts().items()
temp = {}
for k, v in popn_df:
    temp.update({k:v})
sorted(temp.items(), key=lambda x: x[0], reverse=True)
    

[('500', 5741),
 ('50', 1829),
 ('30', 804),
 ('20', 592),
 ('1000+', 1690),
 ('1000', 3947),
 ('100', 16391),
 ('1', 7025),
 ('0', 21381)]

In [214]:
# good 'category' bins look like 0, 1, 2-20, 20-30, 30-50, 50-100, 100-200, 200-300, 300-500, 500-1000, > 1000
# can do that after. Method will be create a category with a string name

In [225]:
# function to create bins for population variable
def popn_bins(population):
    if population == 0:
        return '0'
    elif population == 1:
        return '1'
    elif 2 <= population <  20:
        return '20'
    elif 20 <= population <  30:
        return '30'
    elif 30 <= population <  50:
        return '50'
    elif 50 <= population <  100:
        return '100'
    elif 100 <= population <  200:
        return '100'
    elif 200 <= population <  300:
        return '100'
    elif 300 <= population <  500:
        return '500'
    elif 500 <= population <  1000:
        return '1000'
    else:
        return '1000+'

# for item in df.population:
#     print(item, type(item))
# run function on df
df.population = df.population.apply(popn_bins)


TypeError: '<=' not supported between instances of 'int' and 'str'

In [216]:
df.head()

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,71096,25/03/13,long_rainy,1542,Government,37.894287,-4.363519,Pangani,3,100,...,1978,gravity,user-group,annually,soft,insufficient,spring,groundwater,communal standpipe,functional
1,53061,26/02/11,short_dry,1,Da,38.847129,-6.545266,Wami / Ruvu,6,20,...,2001,mono,commercial,never pay,soft,enough,river/lake,surface,communal standpipe,functional
2,22230,25/03/11,long_rainy,8,Community,36.422095,-6.225916,Wami / Ruvu,1,0,...,1992,gravity,user-group,never pay,soft,enough,spring,groundwater,communal standpipe,functional
3,46668,13/03/13,long_rainy,-15,0,39.342677,-6.855024,Wami / Ruvu,7,20,...,2000,mono,other,per bucket,salty,enough,borehole,groundwater,communal standpipe,functional
4,13080,04/07/13,long_dry,1347,DW,36.528006,-3.393788,Internal,2,100,...,2009,gravity,user-group,unknown,soft,insufficient,spring,groundwater,communal standpipe,functional


In [217]:
df.population[:25]

0     100
1      20
2       0
3      20
4     100
5     100
6       1
7       0
8     500
9       0
10    500
11      1
12      0
13      0
14      0
15      1
16      1
17    100
18    500
19      1
20    100
21    100
22    500
23    100
24      0
Name: population, dtype: object

### Installer

In [218]:
# A lot of work on installer needed.
# look at sorting first to visually inspect potential bins
print(df['installer'].value_counts())
installer_df = df['installer'].value_counts().items()
temp1 = {}
for k, v in installer_df:
    temp1.update({k:v})
sorted(temp1.items(), key=lambda x: x[0], reverse=False)
    

DW                         17847
Government                  3103
Community                   1675
DANIDA                      1602
Hesawa                      1379
RWE                         1206
KKKT                        1187
Council                     1166
one_off                     1098
Fin Water                    972
0                            777
TCRS                         707
CES                          610
TASAF                        502
TWE                          436
LGA                          408
World vision                 408
WEDECO                       397
Gover                        383
Japan Government             355
UNICEF                       332
OXFAM                        330
AMREF                        329
WU                           301
Dmdd                         287
ACRA                         278
ADRA                         276
World Vision                 270
SEMA                         249
Kilwa company                244
          

[('-', 3),
 ('0', 777),
 ('AAR', 4),
 ('ABASIA', 29),
 ('AC', 2),
 ('ACORD', 3),
 ('ACRA', 278),
 ('ACTIVE TANK CO LTD', 11),
 ('AD', 14),
 ('ADAP', 4),
 ('ADB', 5),
 ('ADP', 33),
 ('ADRA', 276),
 ('AF', 4),
 ('AFRICA', 2),
 ('AFRICA MUSLIM', 3),
 ('AFRICAN DEVELOPMENT FOUNDATION', 6),
 ('AI', 105),
 ('AIMGOLD', 9),
 ('AMP Contract', 21),
 ('AMREF', 329),
 ('APM', 2),
 ('AQAL', 2),
 ('AQUARMAN DRILLERS', 2),
 ('ASDP', 2),
 ('ATIGH BUILDINGS', 10),
 ('AUSTRALIA', 4),
 ('AUWASA', 10),
 ('Accra', 30),
 ('Action Contre La Faim', 8),
 ('Active MKM', 26),
 ('Adam', 2),
 ('Af', 25),
 ('Africa', 4),
 ('Africa Amini Alama', 2),
 ('Africare', 2),
 ('Amboni Plantation', 4),
 ('American', 116),
 ('Anglican Church', 135),
 ('Ar', 2),
 ('Arabs Community', 2),
 ('Ardhi', 13),
 ('Artisan', 139),
 ('Athumani  Janguo', 2),
 ('Atlas Company', 5),
 ('BENGUKA', 13),
 ('BESADA', 26),
 ('BFFS', 5),
 ('BGM', 21),
 ('BILILA', 5),
 ('BR', 8),
 ('BRUDER', 16),
 ('BSF', 35),
 ('Baric', 7),
 ('Belgiam Government',

In [219]:
# test
test_dict = {'Arisan': {'Arisan', 'Arian', 'Artisan'}, 'Government': {'Government', 'government', 'govt.'}, 'Japan': {'Japan', 'Japan Government'}}

def allocator(name):
    for key, value in test_dict.items():
        temp = []
        for value1 in value:
            temp.append(value1)
        if name in temp:
            return key
        
    return name
        
print(allocator('Arian'))
print(allocator('Cheese'))
print(allocator('govt.'))
print(allocator('Henry'))
print(allocator('Japan'))

Arisan
Cheese
Government
Henry
Japan


In [220]:
# create installer dictionary (Got as far as T.)
installer_dict = {'ACTIVE TANK CO LTD': {'ACTIVE TANK CO LTD','ACTIVE TANK CO'},
                  'ADP': {'ADP', 'ADP Busangi'},
                  'ADRA': {'ADRA', 'ADRA /Government', 'ADRA/Government','Adra','Adra /Community',
                           'Adra/ Community', 'Adra/Community', 'Adrs'},
                  'AFRICA MUSLIM': {'AFRICA MUSLIM', 'Africa Islamic Agency Tanzania'},
                  'ACTION AID': {'ACTION AID', 'Action Aid'},
                  'AI': {'AI','AIC', 'AIC KI','AICT'},
                  'AMP Contract': {'AMP Contract', 'AMP Contracts','AMP contractor'},
                  'AFRICAN DEVELOPMENT FOUNDATION': {'AFRICAN DEVELOPMENT FOUNDATION', 
                                                     'AFRICAN REFLECTIONS FOUNDATION', 'AGRICAN'},
                  'Anglican Church': {'Anglican Church', 'ANGLIKANA CHURCH', 'ANGLI', 'ANGRIKANA', 'Angli', 
                                      'Anglican', 'Anglican Uganda', 'Anglican church', 'Anglikan',
                                      'Anglikana', 'Angrikana', 'Chur', 'Church', 'CHRISTIAN OUTRICH'},
                  'AQUA BLUES ANGELS': {'AQUA BLUES ANGELS', 'AQUA WEL', 'AQUA Wat', 'AQUA Wel'},
                  'Action Contre La Faim': {'Action Contre La Faim', 'Action Contre la Faim'},
                  'Active MKM': { 'Active MKM', 'Active KMK'},
                  'Islamic': {'Islamic', 'Africa Islamic Agency Tanzania', 'Africa M', 'Africa Muslim Agenc',
                              'African Muslims Age', 'African Realief Committe of Ku', 'Kuwait', 'KUWAIT'},
                  'Africaone': {'Africaone', 'Africaone Ltd'},
                  'Amboni Plantation': {'Amboni Plantation', 'Amboni plantation'},
                  'American': {'American', 'Amec','Amref', 'Amec'},
                  'Arab Community': {'Arab Community', 'Arab community'},
                  'Ardhi': {'Ardhi Instute', 'Ardhi Water Wells', 'Ardhi and PET Companies', 'Ardhi water well'},
                  'CARITAS': {'CARITAS', 'CARTAS', 'CARTAS Tanzania'},
                  'Colonial Government': {'Colonial Government', 'British colonial government'},
                  'Community': {'Community','Commu', 'Communit', 'Comunity','COMMU', 'COMMUNITY'},
                  'Concern': {'Concern', 'Conce', 'Concen', 'Concern /government', 'Concern/Government'},
                  'Consultant': {'Consultant', 'Consultant Engineer', 'Consultant and DWE',
                                'Consulting Engineer', 'Consulting engineer', 'Consuting Engineer'},
                  'Cosmo': {'Cosmo', 'Cosmos Engineering'},
                  'Council': {'Council', 'Counc', 'DISTRICT COUNCIL', 'District  Council','District Counci',
                              'District Council', 'District council', 'Distri'},
                  'Artisan': {'Arisan', 'Arian', 'Artisan', 'Aartisa', 'Atisan'}, 
                  'Babu Sajini': {'Babu Sajini', 'Babu Sajin'},
                  'China': {'CHINA', 'CHINA HENAN CONTRACTOR'},
                  'CJEJOW': {'CJEJOW', 'CJEJ0', 'CJEJOW CONSTRUCTION'},
                  'Concern': {'Concern', 'Concen'},
                  'DANIDA' : {'DANIDA','DANIAD', 'DANID','DANIDS'},
                  'DASIP' : {'DASIP', 'DASP', 'DASSIP'},
                  'DBSPE': {'DBSPE', 'DBSP'},
                  'DW': {'DW', 'DW#', 'DWE & LWI', 'DWR', 'DWW', 'DWE', 'DWE/','DWE}', 'District Water Department'
                        'District water depar', 'District water department', 'District Water Department', 'Distric Water Department', 
                                                'District water department'},
                  'Dr. Matobola': {'Dr. Matobola', 'Dr.Matobola'},
                  'Dr. Matomola': {'Dr. Matomola','Dr.Matomola'},
                  'EL': {'EL', 'ELCT'},
                  'Egypt': {'People from Egypt', 'Tanz Egypt technical coopera', 'Tanzania/ Egypt', 
                            'Wizra ya maji na egypt'},
                  'Fin Water': {'Fin Water', 'F', 'FIN WATER', 'FINI WATER', 'FINI Water','FW', 
                                'Fin water', 'FinW','FinWate','FinWater', 'Fini Water', 
                                'Fini water', 'Finland Government', 'Finwater', 'FINLAND','FINN WATER'},
                  'Government': {'Government', 'GOVERNMENT', 'Government and Community', 'CENTRAL GOVERNMENT',
                                 'Cebtral Government', 'Cental Government', 'Centr', 'Central govt',
                                 'Central government', 'GOVER', 'GOVERM', 'GOVERN', 'GOVERNME', 
                                 'Wizara  ya maji', 'Wizara ya maji', 'wizara ya maji'},
                  'GREINAKER': {'GREINAKER', 'GREINEKER'},
                  'GRA' : {'GRA','GRA TZ MUSOMA'},
                  'GRUMETI':{'GRUMETI', 'GRUMET', 'GRUMETI SINGITA', 'GURUMETI SAGITA CO'},
                  'Grobal resource alliance': {'Grobal resource alliance', 'Grobal resource  alliance'},
                  'Gwasco': {'Gwasco', 'Gwasco L','Gwaseco'},
                  'HAPA': {'HAPA', 'HAPA SINGIDA'},
                  'Hesawa': {'Hesawa', 'HESAWA', 'Hasawa','Hesawz', 'Hesewa'},
                  'Individuals': {'Individuals', 'INDIVIDUALS', 'INDIVIDUAL'},
                  'India': {'India', 'Indi'},
                  'Institution': {'Institution', 'Insititutiona'},
                  'ISF': {'ISF', 'ISF/Government', 'ISF/TACARE'},

                  'Japan Government': {'Japan Government', 'JAPAN EMBASSY', 'JICA','JAICA', 'Jaica', 'JSICA','Jika'
                                      'JAICA CO', 'Embasy of Japan in Tanzania'},
                  'Jeshi la Wokovu': {'Jeshi la Wokovu', 'Jeshi la wokovu', 'Jeshi la wokovu [cida]'},
                  'JUIN': {'JUIN', 'JUIN CO'},
                  'KARUMBA BUILDING COMPANY LTD': {'KARUMBA BUILDING COMPANY LTD', 'KARUMBA BIULDIN',
                                                  'KARUMBA BIULDING COMPANY LTD','KARUMBA BIULDING CONTRACTOR'},
                  'Kilwa company': {'Kilwa company', 'Kilwater', 'Kiliwater'},
                  'KKKT': {'KKKT', 'KK', 'KKKT CHURCH', 'KKKT DME', 'KKKT _ Konde and DWE', 'KKKT-Dioces ya Pare',
                           'KKT', 'KKT C'},
                  'KOBERG': {'KOBERG', 'KOBERG Contractor'},
                  'KYASHA ENTERPR': {'KYASHA ENTERPR', 'KYASHA ENTREPR'},
                  'Lawate fuka water su': {'Lawate fuka water su', 'Lawatefuka water sup'},
                  'Local': {'Local', 'Local  technician', 'Local te', 'Local technical tec', 'Local technician', 
                           'Locall technician'},
                  'LWI': {'LWI', 'LWI &CENTRAL GOVERNMENT'},
                  'MACK DONALD CO LTD': {'MACK DONALD CO LTD','MACK DONALD CONTRACTOR', 'MACK DONALD CONTRSCTOR'},
                  'MKONGO CONSTRUCTION': {'MKONGO CONSTRUCTION', 'MKON CONSTRUCTION', 'MKONG CONSTRUCTION',
                                         'MKONGO BUILDING CONTRACTOR'},
                  'MLAKI  CO': {'MLAKI  CO', 'MLAKI CO'},
                  'MTUWASA': {'MTUWASA', 'MTUWASA and Community'},
                  'MWAKI CONTRACTOR': {'MWAKI CONTRACTOR', 'MWAKI CONTRACTO'},
                  'Makonde': {'Makonde', 'Makonde water Population', 'Makonde water population','Makonde water supply'},
                  'Maswi company': {'Maswi company', 'Maswi'},
                  'Milenium': {'Milenium','Milenia', 'Mileniam', 'Mileniam project'},
                  'Magadini Makiwaru wa': {'Magadini Makiwaru wa', 'Magadini-Makiwaru wa'},
                  'MWE' : {'MWE', 'MWE &'},
                  'MTUWASA': {'MTUWASA','MTUWASA and Community', 'MUWASA','MUWSA'},
                  'OXFAM': {'OXFAM', 'OXFARM'},
                  'NORAD': {'NORAD', 'NIRAD', 'NORA'},
                  'NYAKILANGANI': {'NYAKILANGANI', 'NYAKILANGANI CO', 'NYAKILANGANI CONSTRUCTION'},
                  'Naishu': {'Naishu Construction Co. ltd', 'Naishu construction co. ltd'},
                  'Private': {'Private', 'Privat'},
                  'RC church': {'RC church', 'R.C', 'RC CATHORIC', 'RC/Mission', 'Roman', 'Cathoric', 'rc ch'},
                  'Region Water Department': {'Region Water Department', 'Region water', 'Region water Department', 
                                              'Regional Water'},
                  'SAXON BUILDING CONTRACTORS': {'SAXON BUILDING CONTRACTORS', 'SAXON  BUILDING CONTRACTOR',
                                                 'SAXON BUILDING CONTRACTOR'},
                  'TASAF': {'TASAF', 'TASA', 'TASAF 1', 'TASAF and Comunity', 'TASAF and MMEM', 
                            'TASAFcitizen and LGA', 'TASF', 'TASSAF', 'Tasaf', 'Tasaf and Lga',
                            'TASAF/', 'TASSAF /TCRS', 'TASAF/DMDD'},
                  'Taes': {'Taes', 'Taees'},
                  'TUKWALE ENTERP': {'TUKWALE ENTERP', 'TUKWARE ENTERP'},
                  'TWE': {'TWE', 'TWESA', 'TWESA /Community', 'TWESA/ Community'},         
                  'UMOJA DRILLING': {'UMOJA DRILLING', 'UMOJA DRILLING CONSTRUCTION',
                                     'UMOJA DRILLING CONTRACTOR','UMOJA DRILLING CONTRUCTO'},
                  'UNICEF': {'UNICEF', 'UNICRF','Unicef','Unisef'},
                  'VC': {'VC', 'VCIF'},
                  'VIFAFI': {'VIFAFI', 'VIFAI'},
                  'VITECOS': {'VITECOS','VITECOS INVEST', 'VTECOS'},
                  'Village Govt': {'Village Govt','Village government', 'Village govt', 
                                   'Village local contractor', 'Village water committee', 'Subvillage'},
                  'WORLD BANK': {'WORLD BANK', 'W', 'W/', 'WORLD NK', 'WOULD BANK', 'World bank'},
                  'Water users Group': {'Water use Group','Water user Group','Water users Group'},
                  'Zao': {'Zao','Zao water spring','Zao water spring X'}
                 }

print(len(installer_dict))

92


In [221]:
# test with name of dictionary included. (Might be needed for actual .apply(...))
test_dict2 = {'Arisan': {'Arisan', 'Arian', 'Artisan'}, 'Government': {'Government', 'government', 'govt.'}, 'Japan': {'Japan', 'Japan Government'}}

def allocator2(name, names_dict):
    for key, value in names_dict.items():
        temp = []
        for value1 in value:
            temp.append(value1)
        if name in temp:
            return key
        
    return name
        
print(allocator2('Arian', test_dict2))
print(allocator2('Cheese', test_dict2))
print(allocator2('govt.', test_dict2))
print(allocator2('Henry', test_dict2))
print(allocator2('Japan', test_dict2))

Arisan
Cheese
Government
Henry
Japan


Got from 466 categories to 50, much better!

Next is to turn all of the remaining 1s (and 2s) into another category.  Needs a function to apply after the above function.

The below will work on the df column with apply method

In [222]:
def change_name(name):
    if test_df.installer.value_counts()[name] < 3:
        return 'one-offs'
    else:
        return name
test_df.installer = test_df.installer.apply(change_name)
test_df

NameError: name 'test_df' is not defined

In [223]:
def change_name2(name, df):
    if df.installer.value_counts()[name] < 3:
        return 'one-offs'
    else:
        return name
# test_df.installer = test_df.installer.apply(lambda x: change_name2(x, test_df))
# test_df

Now run installer functions

### Here we reduce number of installer categories

In [124]:
df.installer = df.installer.apply(lambda name: allocator2(name, installer_dict))
df['installer'].value_counts()[:15]

DW            17847
Government     3103
Community      1675
DANIDA         1602
Hesawa         1379
RWE            1206
KKKT           1187
Council        1166
one_off        1098
Fin Water       972
0               777
TCRS            707
CES             610
TASAF           502
TWE             436
Name: installer, dtype: int64

## How to create counts in pandas properly

In [125]:
from collections import Counter
number_installed  = []
for index in df['installer'].value_counts():
    number_installed.append(index)
count_installed = pd.DataFrame.from_dict(Counter(number_installed), orient="index")
count_installed
# ^ count_installed is in format: col0 = how many have you installed; col1 = number of orgs who have installed this many

,0
17847,1
3103,1
1675,1
1602,1
1379,1
1206,1
1187,1
1166,1
1098,1
972,1


Looks reasonable to have a category of one_off if you installed only one water point

In [136]:
one_offs = []
names = df['installer'].value_counts().index
for name in names:
    if df['installer'].value_counts()[name] == 1:
        one_offs.append(name)
len(one_offs)
print(one_offs[:10])

[]


In [137]:
'Islamic' in df.installer.values

True

In [138]:
def is_one_off(name):
    if name in one_offs:
        return 'one_off'
    else: 
        return name

df.installer = df.installer.apply(is_one_off)
'Islamic' in df.installer.values

True

In [139]:
'one_off' in df.installer.values

True

In [135]:
df.installer.values_counts()

AttributeError: 'Series' object has no attribute 'values_counts'

^ So that worked.

Rerun eda_helper

In [ ]:
print(eda_helper(df))

In [ ]:
# save this good data!
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data02.csv', index=False)

# DO AGAIN!

In [360]:
# need to do wpt_year again
df = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data03.csv')

In [361]:
# key is to cahnge construction year from an int to a float (otherwise rounds)
df.construction_year = df.construction_year.astype(np.float64)

In [362]:
type(df.construction_year[1])

numpy.float64

In [364]:
df.head()

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,20163,17/02/13,short_dry,330,DANIDA,37.351138,-11.649440,Ruvuma / Southern Coast,10,1,...,1994.0,swn 80,user-group,annually,soft,insufficient,borehole,groundwater,hand pump,functional needs repair
1,42800,17/02/13,short_dry,327,DANIDA,37.351869,-11.648378,Ruvuma / Southern Coast,10,1,...,1994.0,swn 80,user-group,annually,soft,enough,borehole,groundwater,hand pump,functional needs repair
2,57360,14/02/13,short_dry,439,TASAF,37.082702,-11.586297,Ruvuma / Southern Coast,10,1,...,2010.0,swn 80,user-group,never pay,soft,insufficient,borehole,groundwater,hand pump,non functional
3,338,23/02/13,short_dry,559,MKONGO CONSTRUCTION,36.463786,-11.568577,Ruvuma / Southern Coast,10,500,...,2011.0,other,unknown,unknown,soft,unknown,borehole,groundwater,other,non functional
4,51453,23/02/13,short_dry,568,DWE,36.465277,-11.566805,Ruvuma / Southern Coast,10,100,...,2003.0,swn 80,user-group,monthly,soft,dry,shallow well,groundwater,hand pump,non functional


In [365]:
type(df['date_recorded'][1])

str

In [366]:
df.columns

Index(['id', 'date_recorded', 'season_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'region_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type_group', 'management_group', 'payment_type',
       'water_quality', 'quantity_group', 'source_type', 'source_class',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [ ]:
# Apply wpt_age globally

In [367]:
df.iat[0, 1]

'17/02/13'

In [368]:
type(df.iat[0, 1])

str

In [369]:
# create helper function and test
# BE CAREFUL - how is is in Excel file and df is different

# ARCHIVE DO NOT DELETE
def year_convert(date_recorded):
    """
    takes custom string (e.g. '18/03/13') and converts to a float
    is not super accurate as assumes all months have 31 days
    
    """
    year = float(date_recorded[6:8])
    if year > 30:
        year += 1900
    else: 
        year += 2000
    month = float(date_recorded[3:5])
    day = float(date_recorded[:2])
    
    return year + ((month - 1) / 12) + (day / (31*12)) 

year_convert('18/03/13')

2013.215053763441

In [370]:
# IMPORTANT CHECK TO USE THIS OR PREVIOUS
def year_convert2(date_recorded):
    """
    takes custom string (e.g. '2013-04-17') and converts to a float
    is not super accurate as assumes all months have 31 days
    
    """
    year = float(date_recorded[:4])
    month = float(date_recorded[5:7])
    day = float(date_recorded[8:10])
    
    return year + ((month - 1) / 12) + (day / (31*12)) 

year_convert2('2013-02-17')

2013.1290322580644

In [371]:
df['date_recorded'] = df['date_recorded'].apply(year_convert)


In [372]:
df.head()

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,20163,2013.129032,short_dry,330,DANIDA,37.351138,-11.649440,Ruvuma / Southern Coast,10,1,...,1994.0,swn 80,user-group,annually,soft,insufficient,borehole,groundwater,hand pump,functional needs repair
1,42800,2013.129032,short_dry,327,DANIDA,37.351869,-11.648378,Ruvuma / Southern Coast,10,1,...,1994.0,swn 80,user-group,annually,soft,enough,borehole,groundwater,hand pump,functional needs repair
2,57360,2013.120968,short_dry,439,TASAF,37.082702,-11.586297,Ruvuma / Southern Coast,10,1,...,2010.0,swn 80,user-group,never pay,soft,insufficient,borehole,groundwater,hand pump,non functional
3,338,2013.145161,short_dry,559,MKONGO CONSTRUCTION,36.463786,-11.568577,Ruvuma / Southern Coast,10,500,...,2011.0,other,unknown,unknown,soft,unknown,borehole,groundwater,other,non functional
4,51453,2013.145161,short_dry,568,DWE,36.465277,-11.566805,Ruvuma / Southern Coast,10,100,...,2003.0,swn 80,user-group,monthly,soft,dry,shallow well,groundwater,hand pump,non functional


In [373]:
print(type(df.iat[0, 1]))

<class 'numpy.float64'>


In [374]:
print(type(df.iat[0, 13]))

<class 'numpy.float64'>


In [375]:
df.iat[7, 1] - df.iat[7, 13]

12.14247311827944

In [376]:
df.columns

Index(['id', 'date_recorded', 'season_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'region_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type_group', 'management_group', 'payment_type',
       'water_quality', 'quantity_group', 'source_type', 'source_class',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [377]:
# need another helper function in the case that construction_year is still zero
def check_not_zero(df):
    """ 
    passes if not zero
    else sets to recorded date set as a float
    """
    for i in df.index:
        if df.iat[df.index[i], 13] != 0: 
           pass
            
        else: 
            df.iat[i, 13] = df.iat[i, 1]
                       
    return df

In [378]:
type(df.date_recorded[1])

numpy.float64

In [379]:
df = check_not_zero(df)
print(df.construction_year.value_counts())
# df2.head()
# print(type(df2.construction_year[1]))
# print(type(df2.date_recorded[1]))

1982.000000    3366
2012.000000    2958
2009.000000    2850
1992.000000    2416
1985.000000    2399
2006.000000    2284
2007.000000    2224
2010.000000    2159
1978.000000    2112
2008.000000    1974
2011.000000    1973
1988.000000    1764
1994.000000    1522
1995.000000    1441
1998.000000    1401
1993.000000    1377
2002.000000    1340
2000.000000    1326
1990.000000    1211
1983.000000    1174
1999.000000    1151
1977.000000    1144
2004.000000    1106
2005.000000    1101
1980.000000     947
1986.000000     883
1987.000000     860
1979.000000     762
1996.000000     746
2003.000000     716
               ... 
2011.653226      10
2011.626344      10
2011.629032      10
2012.784946       9
2013.045699       9
2012.830645       9
2012.959677       9
2011.618280       8
2011.639785       8
1967.000000       8
2013.107527       7
2011.637097       7
2011.650538       7
2012.981183       7
2012.973118       6
2012.978495       5
2012.825269       5
2011.645161       5
2013.174731       4


In [380]:
df['construction_year'][:300]

0      1994.0
1      1994.0
2      2010.0
3      2011.0
4      2003.0
5      2007.0
6      2007.0
7      2001.0
8      1994.0
9      1998.0
10     2003.0
11     2003.0
12     2007.0
13     2003.0
14     2007.0
15     2007.0
16     2003.0
17     2008.0
18     2006.0
19     2006.0
20     2006.0
21     2006.0
22     2006.0
23     2006.0
24     1994.0
25     1987.0
26     1988.0
27     1988.0
28     1988.0
29     1988.0
        ...  
270    2007.0
271    2007.0
272    1992.0
273    1992.0
274    2007.0
275    2012.0
276    1992.0
277    2012.0
278    2011.0
279    1996.0
280    2004.0
281    2004.0
282    1990.0
283    2012.0
284    2012.0
285    2012.0
286    1990.0
287    2008.0
288    2008.0
289    2004.0
290    2008.0
291    1992.0
292    2007.0
293    1990.0
294    1990.0
295    1990.0
296    2008.0
297    2004.0
298    1994.0
299    1994.0
Name: construction_year, Length: 300, dtype: float64

In [382]:
df.columns

Index(['id', 'date_recorded', 'season_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'region_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'construction_year',
       'extraction_type_group', 'management_group', 'payment_type',
       'water_quality', 'quantity_group', 'source_type', 'source_class',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [ ]:
# worked - need to now create wpt age

In [383]:
df_wpt_age = df.date_recorded - df.construction_year
# df.insert(13,'wpt_age',df_wpt_age )

In [384]:
df_wpt_age.head()

0    19.129032
1    19.129032
2     3.120968
3     2.145161
4    10.145161
dtype: float64

In [385]:
df.insert(13,'wpt_age', df_wpt_age )

In [386]:
df.columns

Index(['id', 'date_recorded', 'season_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'region_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'wpt_age',
       'construction_year', 'extraction_type_group', 'management_group',
       'payment_type', 'water_quality', 'quantity_group', 'source_type',
       'source_class', 'waterpoint_type_group', 'status_group'],
      dtype='object')

In [389]:
df['wpt_age'][58800: 59000]

58800    0.0
58801    0.0
58802    0.0
58803    0.0
58804    0.0
58805    0.0
58806    0.0
58807    0.0
58808    0.0
58809    0.0
58810    0.0
58811    0.0
58812    0.0
58813    0.0
58814    0.0
58815    0.0
58816    0.0
58817    0.0
58818    0.0
58819    0.0
58820    0.0
58821    0.0
58822    0.0
58823    0.0
58824    0.0
58825    0.0
58826    0.0
58827    0.0
58828    0.0
58829    0.0
        ... 
58970    0.0
58971    0.0
58972    0.0
58973    0.0
58974    0.0
58975    0.0
58976    0.0
58977    0.0
58978    0.0
58979    0.0
58980    0.0
58981    0.0
58982    0.0
58983    0.0
58984    0.0
58985    0.0
58986    0.0
58987    0.0
58988    0.0
58989    0.0
58990    0.0
58991    0.0
58992    0.0
58993    0.0
58994    0.0
58995    0.0
58996    0.0
58997    0.0
58998    0.0
58999    0.0
Name: wpt_age, Length: 200, dtype: float64

In [ ]:
### What has happened is that with rounding, zero construction 

In [390]:
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data04.csv', index=False)

In [171]:
def eda_helper(df):
   dict_list = []
   for col in df.columns:
       data = df[col]
       dict_ = {}
       # The null count for a column. 
       dict_.update({"null_count" : data.isnull().sum()})
       # Counting the unique values in a column
       dict_.update({"unique_count" : len(data.unique())})
       # Finding the types of data in the column
       # This is useful for finding out potential problems with type mismatches
       dict_.update({"data_type" : set([type(d).__name__ for d in data])})
       #dict_.update({"score" : match[1]})
       dict_list.append(dict_)
   eda_df = pd.DataFrame(dict_list)
   eda_df.index = df.columns
   eda_df.sort_values(by=['null_count', 'unique_count'], ascending=[True, False], inplace=True)
       
   return eda_df

In [172]:
print(eda_helper(df))

                           data_type  null_count  unique_count
id                             {int}           0         59400
latitude                     {float}           0         57517
longitude                    {float}           0         57516
gps_height                     {int}           0          2429
date_recorded                  {str}           0           356
construction_year              {int}           0            55
region_code                    {int}           0            27
extraction_type_group          {str}           0            13
basin                          {str}           0             9
population                     {str}           0             9
water_quality                  {str}           0             8
payment_type                   {str}           0             7
source_type                    {str}           0             7
waterpoint_type_group          {str}           0             6
management_group               {str}           0       

In [179]:
def year_to_str(year):
    if year < 2000:
        year_str = str(year)[-2:]
        year_19 = 
    else:
        year_str = str(year)[-2:]
    str_a = str(year)
    output = '01/01/' 
    return output

# a0 = year_to_str(2014)
# print(a0, type(a0))


2014/00/00 <class 'str'>


In [189]:
year = 2014
a = str(year)[-2:]
print(a, type(a))

14 <class 'str'>


In [181]:
df_wpt_age = df['date_recorded'].apply(year_convert) - df['construction_year'].apply(check_not_zero)
df_wpt_age.head()

ValueError: could not convert string to float: '25/0'

In [130]:
df.insert(14, 'wpt_age', df_wpt_age)
df.head()

NameError: name 'df_wpt_age' is not defined

In [ ]:
# save this update
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/corrected_data03.csv', index=False)

In [ ]:
categories = {'extraction_type_group', 'region_code', 'population', 'water_quality', 'payment_type',
              'source_type', 'basin', 'management_group', 'quantity_group', 'season_recorded', 
              'waterpoint_type_group', 'source_class', 'status_group', 'permit', 'installer', 
              'public_meeting', 'scheme_management', 'region_code'
            }

for item in categories:
    df[item] = df[item].astype('category')

In [ ]:
print(eda_helper(df))

In [ ]:
ax.set_xlabel('X_axi',fontsize=0.1)
ax = sns.countplot(x='extraction_type_group', hue='status_group', data=df)
ax.figure.savefig("output0.png", dpi = 1000)

In [ ]:
ax = sns.countplot(x='region_code', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='population', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='water_quality', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='payment_type', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='source_type', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='basin', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='management_group', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='quantity_group', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='season_recorded', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='waterpoint_type_group', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='source_class', hue='status_group', data=df)

In [ ]:
ax = sns.countplot(x='permit', hue='status_group', data=df)

In [ ]:
ax.set_xlabel('X_axi',fontsize=0.2)
ax = sns.countplot(x='installer', hue='status_group', data=df)
ax.figure.savefig("output1.png", dpi = 800)

In [ ]:
ax.set_xlabel('X_axi',fontsize=0.4)

sns.axes_style()
# sns.set_context("notebook", font_scale=.5, rc={"lines.linewidth": 2.5})
ax = sns.countplot(x='scheme_management', hue='status_group', data=df)
ax.figure.savefig("output1.png", dpi = 400)

In [ ]:
ax.figure.savefig("output.png", dpi = 400)

In [ ]:
ax = sns.countplot(x='public_meeting', hue='status_group', data=df)

In [ ]:
df.groupby('installer')['status_group'].value_counts()

In [ ]:
# for r in range([start], stop[, step])
for r in range(1000, 1100, 1): print(r)